In [1]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !pip install -q transformers==4.48.2 einops accelerate bitsandbytes

## Importando as seguintes bibliotecas
* <b>AutoModelForCausalLM:</b> Utilizado para carregar um modelo pré-treinado para gerar texto (escrever histórias, responder perguntas, completar código, etc.) e não quer se preocupar em encontrar a classe específica para aquele modelo.
* <b>AutoTokenizer:</b> Uma classe que fornece um tokenizador que corresponde ao modelo. O tokenizador é responsável por converter texto em tokens (numéricos) que o modelo pode entender.
* <b>pipeline:</b> fornece uma interface simples e unificada para várias tarefas de PLN, facilitando a execução de tarefas como geração de texto, classificação e tradução.
* <b>BitsAndBytesConfig:</b> Uma classe para configuração de quantização e outras otimizações de baixo nível para melhorar a eficiência computacional.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from keys import tokens
import torch
import getpass

ImportError: /home/mborges/miniconda3/envs/IA/lib/python3.11/site-packages/torch/lib/libtorch_cpu.so: undefined symbol: iJIT_NotifyEvent

## Leitura da chave do Hugging Face e verificando se o cuda está ativo

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
T = tokens()
print(device,",",T)

>>>TOKEN ACESSADO COM SUCESSO<<
cuda , hf_ozQeB


%env

In [3]:
torch.random.manual_seed(42)

## Carregando o modelo

In [4]:
id_model = "microsoft/Phi-3-mini-4k-instruct"

model = AutoModelForCausalLM.from_pretrained(
    id_model,            
    device_map = device,        # Para acelerar o processo, o modelo usará a placa de vídeo (GPU) que o coputador oferece ou mesmo CPU.
    torch_dtype = "auto",       # Define o tipo de dado ideal para o modelo rodar mais rápido e usar menos memória.
    trust_remote_code = True,   # Permite usar código personalizado do Hugging Face, necessário para modelos específicos.
    attn_implementation="eager" # Escolhe um método de atenção ("eager") para melhorar o desempenho de alguns modelos.
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install 'accelerate>=0.26.0'`

##  Criando o Tokinizador 

<p>O tokenizador traduz o texto em números (tokens) para que o modelo de IA consiga processá-lo.</br> Em seguida, ele converte os números gerados pelo modelo de volta em texto que nós podemos entender.</p>

In [6]:
tokenizer = AutoTokenizer.from_pretrained(id_model)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

## Criação do Pipeline

Parâmetros:

* "text-generation": especifica a tarefa que o pipeline está configurado para executar. Neste caso, estamos configurando um pipeline para geração de texto. O pipeline usará o modelo para gerar texto com base em um     prompt fornecido.
* model=model: especifica o modelo pré-treinado que o pipeline usará. Aqui, estamos passando o model que carregamos anteriormente. Este modelo é responsável por gerar texto com base nos tokens de entrada.
* tokenizer=tokenizer: especifica o tokenizador que o pipeline usará. Passamos o tokenizer que carregamos anteriormente para garantir que o texto de entrada seja tokenizado corretamente e os tokens de saída sejam     decodificados com precisão.

In [7]:
pipe = pipeline(
    "text-generation",
    model = model, 
    tokenizer= tokenizer
)

Device set to use cuda


In [7]:
type(pipe)

transformers.pipelines.text_generation.TextGenerationPipeline

## Parâmetros para geração de texto

Para personalizar o comportamento do nosso pipeline de geração de texto, podemos passar um <b>dicionário de argumentos</b> para controlar vários aspectos do processo de geração.

`max_new_tokens` - Este parâmetro especifica o número máximo de novos tokens a serem gerados em resposta ao prompt de entrada. Ele controla o comprimento do texto gerado.
* Exemplo: Definir max_new_tokens como 500 significa que o modelo gerará até 500 tokens além do prompt de entrada.

`return_full_text` - Determina se deve retornar o texto completo, incluindo o prompt de entrada, ou apenas os tokens recém-gerados.
* Exemplo: Definir return_full_text como False significa que apenas os tokens recém-gerados serão retornados, excluindo o prompt de entrada original. Se definido como True, o texto retornado incluirá o prompt de entrada e a continuação gerada.

`temperature` - Controla a aleatoriedade do processo de geração de texto. Valores mais baixos tornam a saída do modelo mais determinística e focada, enquanto valores mais altos aumentam a aleatoriedade e a criatividade.
* Exemplo: Uma temperatura de 0,1 torna as previsões do modelo mais confiáveis e menos variadas, levando a saídas mais previsíveis. Uma temperatura mais alta resultaria em um texto mais diverso e variado.

`do_sample` - Este parâmetro habilita ou desabilita a amostragem (sampling) durante a geração de texto. Quando definido como True, o modelo faz a amostragem de tokens com base em suas probabilidades, adicionando um elemento de aleatoriedade à saída. Quando definido como False, o modelo sempre escolhe o token de maior probabilidade (decodificação gananciosa / greedy decoding).
* Exemplo: Definir do_sample como True permite uma geração de texto mais diversa e criativa. Se definido como False, a saída será mais determinística, mas potencialmente menos interessante.


In [8]:
generation_args = {
    "max_new_tokens": 1024,
    "return_full_text": False,
    "temperature": 0.1, # 0.1 até 0.9
    "do_sample": True,
}

In [9]:
type(generation_args)

dict

In [9]:
# prompt = "Explique o que é computação quântica"
prompt = "Quanto é 7 x 6 - 42?"

output = pipe(prompt, **generation_args);

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
You are not running the flash-attention implementation, expect numerical differences.


In [10]:
output, type(output)

([{'generated_text': '\n\nOpções de resposta: (A) 0 (B) 1 (C) 2 (D) 4 (E) 6\n\n\n### Answer\n\nPara resolver a expressão 7 x 6 - 42, precisamos seguir a ordem das operações, que é frequentemente lembrada pelo acrônimo PEMDAS (Parênteses, Expoentes, Multiplicação e Divisão, e Adição e Subtração). Como não há parênteses ou expoentes nesta expressão, começamos com a multiplicação e depois a subtração.\n\nPasso 1: Realizar a multiplicação\n7 x 6 = 42\n\nPasso 2: Subtrair o resultado da multiplicação de 42\n42 - 42 = 0\n\nPortanto, a resposta para a expressão 7 x 6 - 42 é 0.'}],
 list)

In [11]:
print(output[0]['generated_text'], "\n")



Opções de resposta: (A) 0 (B) 1 (C) 2 (D) 4 (E) 6


### Answer

Para resolver a expressão 7 x 6 - 42, precisamos seguir a ordem das operações, que é frequentemente lembrada pelo acrônimo PEMDAS (Parênteses, Expoentes, Multiplicação e Divisão, e Adição e Subtração). Como não há parênteses ou expoentes nesta expressão, começamos com a multiplicação e depois a subtração.

Passo 1: Realizar a multiplicação
7 x 6 = 42

Passo 2: Subtrair o resultado da multiplicação de 42
42 - 42 = 0

Portanto, a resposta para a expressão 7 x 6 - 42 é 0. 



In [12]:
prompt = "Me fale um pouco do Brasil"
output = pipe(prompt, **generation_args)

print(f"{output[0]['generated_text']}\n")

.

### Answer:O Brasil é o país mais populoso da América do Sul e o quarto maior do mundo, com uma população estimada em mais de 210 milhões de pessoas. É conhecido por sua rica biodiversidade, com uma grande variedade de paisagens, desde as florestas tropicais da Amazônia até as planícies do Pantanal e as montanhas da Serra do Espinhaço.

A economia do Brasil é diversificada, com setores importantes incluindo agricultura, mineração, manufatura e serviços. O país é um dos maiores produtores de açúcar, soja, café e algodão, e também é um importante exportador de minérios como ferro, cobre e ouro.

A cultura brasileira é vibrante e influenciada por uma mistura de tradições indígenas, africanas e europeias. A música, como samba, bossa nova e pagode, é mundialmente famosa, e o país também é conhecido por seus desfiles de carnaval, que são celebrados em todo o país.

A política brasileira é caracterizada por um sistema de governo presidencialista, com o presidente como chefe de estado e gov

In [13]:
# Definindo os tokens de paradas
# obs: para cada modeo é aconselhável ver a documentação do modelo e sobre o tokens de parada ( end token )
# Para este modelo vamos utilizar este tokens de parada


# prompt = "Explique o que é computação quântica"

template = """<|system|>
Você é um assistente de ajuda.<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt) # formata o prompt para o formato adequado 

In [14]:
print(template)

<|system|>
Você é um assistente de ajuda.<|end|>
<|user|>
"Me fale um pouco do Brasil"<|end|>
<|assistant|>


In [15]:
template

'<|system|>\nVocê é um assistente de ajuda.<|end|>\n<|user|>\n"Me fale um pouco do Brasil"<|end|>\n<|assistant|>'

In [16]:
output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 O Brasil é o maior país da América do Sul e o quarto maior do mundo em extensão territorial, com uma área de aproximadamente 8.515.767 quilômetros quadrados. Sua capital é Brasília, inaugurada em 1960, e sua maior cidade é São Paulo, conhecida por sua rica história, cultura e economia dinâmica. O país é conhecido por sua diversidade cultural, com mais de 200 grupos indígenas e uma população de mais de 210 milhões de pessoas. O Brasil tem uma economia vibrante e é um dos maiores produtores de açúcar, café, soja e petróleo do mundo. Além disso, o país é famoso por seus festivais, como o Carnaval, que é celebrado em todo o país e é considerado um dos maiores eventos culturais do mundo.


## Explorando a engenharia de prompt

In [17]:
prompt = "Descreva quais são as principais bigtechs do mercado global nos dias tuais."
sys_prompt = "Você é um assistente virtual prestativo. Responda as perguntas em português."

template = """<|system|>
{}<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(sys_prompt, prompt)

print(template)

<|system|>
Você é um assistente virtual prestativo. Responda as perguntas em português.<|end|>
<|user|>
"Descreva quais são as principais bigtechs do mercado global nos dias tuais."<|end|>
<|assistant|>


In [18]:
output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 As principais bigtechs do mercado global são empresas que operam em setores como tecnologia da informação, internet e inteligência artificial, e que têm uma influência significativa em diversos aspectos da economia e da sociedade. Algumas das maiores e mais influentes bigtechs incluem:


1. **Google**: Fundada em 1998, a Google é uma empresa multinacional que desenvolve e vende produtos e serviços relacionados à internet, incluindo navegadores de internet, sistemas operacionais, software de produtividade, e-mail, serviços de busca, entre outros.


2. **Facebook (Meta Platforms)**: Fundada em 2004, o Facebook é uma plataforma de rede social que também oferece serviços como Instagram, WhatsApp e Oculus VR. A Meta Platforms, que é o nome atual da empresa após a mudança de nome em 2021, está focada na realidade virtual e na tecnologia Meta.


3. **Amazon**: Fundada em 1994, a Amazon é uma empresa de comércio eletrônico que oferece uma ampla gama de produtos e serviços, incluindo varejo, s

In [19]:
prompt = "quem formulou a primeira lei da física?"

sys_prompt = "Você é um programador experiente. Retorne o código requisitado e forneça explicações."

template = """<|system|>
{}<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(sys_prompt, prompt)

output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 A primeira lei da física, também conhecida como a lei da conservação da energia, foi formulada por Sir Isaac Newton. A lei afirma que a energia não pode ser criada nem destruída, apenas transformada de uma forma para outra. A expressão matemática da primeira lei pode ser escrita como:


F = ma


Onde F é a força aplicada a um objeto, m é a massa do objeto e a é a aceleração resultante. Essa equação é fundamental para o campo da física e tem implicações em várias áreas, como mecânica clássica, dinâmica de fluidos e até mesmo na física quântica.


# Formato de Mensagens (LLMs em Chat)

**Ideia central:** Em chats com LLMs, a conversa é uma **lista de mensagens**.  
Cada mensagem tem:
- **`role`**: quem está falando (`system`, `user`, `assistant`)
- **`content`**: o texto da mensagem

## Por que usar esse formato?
- O modelo entende **contexto de ida e volta** (vários turnos).
- Facilita controlar o comportamento (ex.: instruções no `system`).

## Estrutura do prompt (lista `msg`)
Você envia algo como uma lista de dicionários com `role` e `content`.  
O modelo responde levando em conta **toda a conversa**, com foco na **última** mensagem.

### Exemplo mínimo
```json
[
  { "role": "system", "content": "Você é um assistente útil e direto." },
  { "role": "user",   "content": "Explique o que é uma mensagem." }
]


In [20]:
prompt = "Explique de uma forma simples o conceito de prompt e tamplet"

msg = [
    {"role": "system", "content": "Você é um assistente virtual prestativo com conhecimentos e praticas de IA. Responda as perguntas em português." },
    {"role": "user", "content": prompt}
]


output = pipe(msg, **generation_args)
print(output[0]["generated_text"])

 Um prompt é uma instrução ou solicitação que você dá a um modelo de IA, como um chatbot ou uma inteligência artificial, para que ele possa entender e responder adequadamente. É como uma dica ou um comando que ajuda o modelo a gerar uma resposta relevante.


Por outro lado, um template é um modelo pré-definido que contém uma estrutura ou formato específico para uma resposta. Ele pode incluir espaços reservados ou marcadores que são substituídos por informações específicas quando o modelo de IA processa o prompt. Templates são úteis para garantir que as respostas sejam consistentes e organizadas de acordo com um padrão específico.


Em resumo, um prompt é uma instrução para o modelo de IA, enquanto um template é um modelo de resposta que ajuda a organizar e padronizar as respostas.


### Otimizando com quantização

🧠 Otimizando com Quantização

Modelos grandes podem ser difíceis de executar em máquinas com poucos recursos.Para contornar isso, usamos quantização, uma técnica que reduz o uso de memória e acelera os cálculos ao representar pesos com menor precisão (peso com int8 ou 4 bits).

Com o BitsAndBytesConfig da biblioteca transformers, é possível carregar modelos massivos de forma eficiente, sem grandes perdas de desempenho.

⚙️ Parâmetros principais

<b>load_in_4bit</b> → Ativa a quantização em 4 bits.
Impacto: menor uso de memória e inferência mais rápida.

<b>bnb_4bit_quant_type</b> → Define o tipo de quantização; "nf4" (NormalFloat4) mantém o desempenho com menor precisão.
Impacto: equilíbrio entre tamanho e performance.

<b>bnb_4bit_use_double_quant</b> → Habilita a quantização dupla.
Impacto: reduz erros e melhora a estabilidade.

<b>bnb_4bit_compute_dtype</b> → Define o tipo de dado usado nos cálculos, como torch.bfloat16.
Impacto: cálculos eficientes com mínima perda de precisão.

🚀 <b><u>Resultado<u></b>

Essas configurações permitem rodar modelos grandes mesmo em hardwares limitados, tornando o processo mais leve e rápido, sem comprometer muito a qualidade do modelo.

<b>Observação:</b> Se o modelo der erro de acesso (<u>Cannot access gated repo for url...</u>), use o seguinte comando no prompt:
   - huggingface-cli login
   - Cole seu token

In [23]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [25]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
                                 model_id, 
                                 quantization_config = quantization_config,
                                 token = token # envia o token para o huggingface dar licensa para baixar o modelo em questão  
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [26]:
# Normalmente, ao preparar dados para o LLM, usa-se tokenizer.apply_chat_template, que adiciona tokens EOS (fim de sequência) após cada resposta.

prompt = "Quem foi Dom Pedro II?"
messages = [
    {"role": "user", "content": prompt},
]

<hr style="border: 2px solid black; width: 100%;">

#### <p>A função  <u>tokenizer.apply_chat_template()</u>  da Hugging Face aplica automaticamente o formato correto de chat para o modelo escolhido, evitando erros manuais. O parâmetro <u>return_tensors="pt"</u> indica que os dados devem ser retornados como tensores PyTorch.</p>

#### <u><b>As demais linhas de código:</br></u> 

#### tokenizam as mensagens de entrada, movem os tensores para o dispositivo correto (GPU), geram novos tokens com base nos inputs fornecidos,</br> decodificam os tokens gerados de volta em texto legível e finalmente retornam o texto gerado.</br>

<hr style="border: 2px solid black; width: 100%;">

### O fluxo geral do código é:
* `model_inputs = encodeds.to(device)` -  Move os tensores codificados para o dispositivo especificado (CPU ou GPU) para serem processados pelo modelo.

* `encodeds` - Os tensores gerados na linha anterior.
`to(device)` -  Move os tensores para o dispositivo especificado (device), que pode ser uma CPU ou GPU.

* `generated_ids = model.generate...` -> Gera uma sequência de tokens a partir dos model_inputs.
     * model.generate: Função do modelo que gera texto baseado nos inputs fornecidos.
     * model_inputs: Os inputs processados, prontos para serem usados pelo modelo.
     * max_new_tokens=1000: Limita a geração a no máximo 1000 novos tokens.
     * do_sample=True: Habilita amostragem aleatória durante a geração, o que pode resultar em saídas mais variadas.
     * pad_token_id=tokenizer.eos_token_id: Define o token de padding para ser o token de fim de sequência, garantindo que a geração seja corretamente terminada.
</br>

* `decoded = tokenizer.batch_decode(generated_ids)` - decodifica os IDs gerados de volta para texto legível.
 * `tokenizer.batch_decode` - função que converte uma lista de IDs de tokens de volta para texto.
 * `generated_ids` - os IDs dos tokens gerados na etapa anterior.


`res = decoded[0]` - extrai o primeiro item da lista de textos decodificados.
decoded[0]: Pega o primeiro texto da lista decoded, que corresponde à geração de texto para o primeiro (e possivelmente único) input fornecido.


In [27]:
encondeds = tokenizer.apply_chat_template(messages, return_tensors = "pt")
model_input = encondeds.to(device)

generated_ids = model.generate(
                      model_input, 
                      max_new_tokens = 500,
                      do_sample = True, 
                      pad_token_id = tokenizer.eos_token_id
)

decoded = tokenizer.batch_decode(generated_ids)
res = decoded[0]
res

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nQuem foi Dom Pedro II?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nDom Pedro II foi o Imperador do Brasil entre 1831 e 1889. Ele nasceu em 2 de dezembro de 1825, em São Cristóvão, na então Capitania de São Cristóvão, no Brasil, e morreu em 5 de janeiro de 1891, em Porto Alegre, Rio Grande do Sul, Brasil.\n\nPedro II foi o segundo Imperador do Brasil, após a abdicação de seu pai, Dom Pedro I, que havia declarado a independência do Brasil em 1822. Dom Pedro II ascendeu ao trono com apenas seis anos de idade, após a morte de seu pai, e foi reinando por mais de 58 anos, até a Proclamação da República Brasileira em 1889.\n\nDurante seu reinado, Dom Pedro II implementou várias reformas políticas, econômicas e sociais no Brasil, incluindo a abolição da escravatura em 1888. Ele também promoveu o desenvolvimento da educação, da ciência e da arte no país.\n\nDom Pedro II era conhecido por sua inteligência, sua cultura e 

In [29]:
print(res)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Quem foi Dom Pedro II?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Dom Pedro II foi o Imperador do Brasil entre 1831 e 1889. Ele nasceu em 2 de dezembro de 1825, em São Cristóvão, na então Capitania de São Cristóvão, no Brasil, e morreu em 5 de janeiro de 1891, em Porto Alegre, Rio Grande do Sul, Brasil.

Pedro II foi o segundo Imperador do Brasil, após a abdicação de seu pai, Dom Pedro I, que havia declarado a independência do Brasil em 1822. Dom Pedro II ascendeu ao trono com apenas seis anos de idade, após a morte de seu pai, e foi reinando por mais de 58 anos, até a Proclamação da República Brasileira em 1889.

Durante seu reinado, Dom Pedro II implementou várias reformas políticas, econômicas e sociais no Brasil, incluindo a abolição da escravatura em 1888. Ele também promoveu o desenvolvimento da educação, da ciência e da arte no país.

Dom Pedro II era conhecido por sua inteligência, sua cultura e sua habilid

In [30]:
type(res)

str